## I. Build Features


In [ ]:
from download import main

In [ ]:
main(date=datetime.datetime.strptime('01/01/2010', '%m/%d/%Y'), export=True)

In [ ]:
import datetime
import optparse
import requests
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sys import exit
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('./data/DOWNLOADED_DATA.csv')

In [ ]:
result_possibilities = ['Pass', 'Fail', 'Pass w/ Conditions']
valid_result_bools = [result in result_possibilities for result in data.results]
data = data[valid_result_bools]

In [ ]:
def fail(result):
    if 'Fail' in result:
        return 1
    else:
        return 0   
    
def grocery_store(facility_type):
    if 'Grocery Store' == facility_type:
        return 1
    else:
        return 0

In [ ]:
data['result_binary'] = [fail(result) for result in data.results]
data['grocery'] = [grocery_store(facility_type) for facility_type in data.facility_type]

In [ ]:
top_inspection_types = data.groupby('inspection_type').count().result_binary.sort_values(ascending=False).index[:10]
top_inspection_types_sorted = data.groupby('inspection_type').mean().result_binary[top_inspection_types].sort_values(ascending=False).index

fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.
width = 0.4

data.groupby('inspection_type').mean().result_binary[top_inspection_types].sort_values(ascending=False).plot(kind='bar',color='r',ax = ax, alpha=0.7,width=width,position=0)
data.groupby('inspection_type').count().result_binary[top_inspection_types_sorted].plot(kind='bar',color='b',ax = ax2, alpha=0.7,width=width,position=1)

In [ ]:
inspection_type_dummies = pd.get_dummies(data['inspection_type'])
inspection_type_dummies = inspection_type_dummies[top_inspection_types]
data = pd.concat([data,inspection_type_dummies],axis=1,join='inner')

In [ ]:
biz_cols = []
for col in data.columns:
    if 'business_activity' in col:
        biz_cols.append(col)
        
count = []
mean = []
for col in biz_cols:
    count.append(data[data[col] == 1].result_binary.mean())
    mean.append(len(data[data[col] == 1].result_binary))

In [ ]:
biz_cols_mean = pd.Series(mean,biz_cols)
biz_cols_count = pd.Series(count,biz_cols)

biz_cols_count_top = biz_cols_count.sort_values(ascending=False).index[:10]


biz_col_mean_sorted = biz_cols_mean[biz_cols_count_top].sort_values(ascending=False).index

fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.
width = 0.4

biz_cols_mean[biz_col_mean_sorted].sort_values(ascending=False).plot(kind='bar',color='r',ax = ax, alpha=0.7,width=width,position=0)
biz_cols_count[biz_col_mean_sorted].plot(kind='bar',color='b',ax = ax2, alpha=0.7,width=width,position=1)

In [ ]:
biz_cols_counts = data[biz_cols].sum().sort_values()

pass_rate = []
count = []
for col in biz_cols:
    pass_rate.append(data[data[col] == 1].result_binary.mean())
    pass_rate.append(data[data[col] == 1].result_binary.mean())    
    
top_n = 20
top_license_counts = biz_cols_counts.ix[biz_cols_counts.index[-top_n:]]

top_biz_counts_cols = list(biz_cols_counts.index[-top_n:])

pass_rate = []
for col in top_biz_counts_cols:
    pass_rate.append(data[data[col] == 1].result_binary.mean())
    
top_biz_pass_rate = pd.Series(data = np.array(pass_rate),index=top_biz_counts_cols)

In [ ]:
pd.DataFrame(1-top_biz_pass_rate).plot(kind='bar')
plt.show()
pd.DataFrame(top_license_counts).plot(kind='bar')
plt.show()

In [ ]:
keep_n=5
all_license_cols = list(biz_cols)
for col in list(biz_cols_counts.index[-keep_n:]):
    all_license_cols.remove(col)

drop_license_cols = all_license_cols
data = data.drop(drop_license_cols,axis=1)

In [ ]:
data['inspection_date'] = pd.to_datetime(data['inspection_date'])
#data['license_start_date'] = pd.to_datetime(data['license_start_date'])
#data['date_issued'] = pd.to_datetime(data['date_issued'])
data['creation_date'] = pd.to_datetime(data['creation_date'])

In [ ]:
# weather_data_ohare = pd.read_csv('Ohare Weather Data.csv')
# weather_data_midway = pd.read_csv('Midway Weather Data.csv')
weather_data_ohare = pd.read_csv('./data/Ohare Weather Data.csv')
weather_data_midway = pd.read_csv('./data/Midway Weather Data.csv')

weather_data_ohare['inspection_date'] = [datetime.datetime.strptime(str(DATE), '%Y%m%d') for DATE in weather_data_ohare.DATE]
weather_data_midway['inspection_date'] = [datetime.datetime.strptime(str(DATE), '%Y%m%d') for DATE in weather_data_midway.DATE]
temp_data_ohare = weather_data_ohare.set_index(['inspection_date'])[['TMAX']]
temp_data_midway = weather_data_midway.set_index(['inspection_date'])[['TMAX']]

temp_data_ohare['TMAX_3DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,3)
temp_data_ohare['TMAX_10DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,10)
temp_data_ohare['TMAX_30DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,30)

temp_data_midway['TMAX_3DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,3)
temp_data_midway['TMAX_10DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,10)
temp_data_midway['TMAX_30DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,30)
data = data.join(temp_data_midway, on='inspection_date')

In [ ]:
import re

critical_flags = re.compile(r'(\bCRITICAL\b)',flags = re.IGNORECASE)
serious_flags = re.compile(r'(\bSERIOUS\b)',flags = re.IGNORECASE)
citation_flags = re.compile(r'(\bCITATION\b)',flags = re.IGNORECASE)
violation_flags = re.compile(r'(\bVIOLATION\b)',flags = re.IGNORECASE)
minor_flags = re.compile(r'(\bMINOR\b)',flags = re.IGNORECASE)
corrected_flags = re.compile(r'(\bCORRECTED\b)',flags = re.IGNORECASE)


In [ ]:
violations_count = []
citations_count = []
critical_count = []
serious_count = []
minor_count = []
corrected_count = []

LOG_EVERY_N = 10000


  
for index in data.index:
    text = data.ix[index].violations
    
    if type(text) == type(' '):
        violations_count.append(violation_flags.subn('',text)[1])
        citations_count.append(citation_flags.subn('',text)[1])
        critical_count.append(critical_flags.subn('',text)[1])
        serious_count.append(serious_flags.subn('',text)[1])
        minor_count.append(minor_flags.subn('',text)[1])
        corrected_count.append(corrected_flags.subn('',text)[1])


    else:
        violations_count.append(float('nan'))
        citations_count.append(float('nan'))
        critical_count.append(float('nan'))
        serious_count.append(float('nan'))
        minor_count.append(float('nan'))
        corrected_count.append(float('nan'))

        
    if (index % LOG_EVERY_N) == 0:
        print index


In [ ]:
data['violations_count'] = violations_count
data['citations_count'] = citations_count
data['critical_count'] = critical_count
data['serious_count'] = serious_count
data['minor_count'] = minor_count
data['corrected_count'] = corrected_count




In [ ]:
# compare pass rates for different violation counts
bins = np.linspace(0,6,7)
data[data.result_binary == 1].citations_count.hist(normed=False,alpha=0.5,bins=bins)
data[data.result_binary == 0].citations_count.hist(normed=False,alpha=0.5,bins=bins)

In [ ]:
def prev_inspection_features(data,inspection_id,date):
    current_inspection = data.ix[data[data.inspection_id==inspection_id].index[0]]
    
    all_inspections = data[data.license_ == current_inspection.license_]
    
    previous_inspections = all_inspections[all_inspections.inspection_date < date]
    if len(previous_inspections) >= 1:
        last_inspection_index = previous_inspections.inspection_date.sort_values().index[-1]
        fraction = previous_inspections.result_binary.mean()
        count = previous_inspections.result_binary.count()
        last = previous_inspections.result_binary[last_inspection_index]
        
        #if type(pd.to_datetime('nan')) != type(current_inspection.license_start_date):
        #    license_age = current_inspection.inspection_date - current_inspection.license_start_date
        #    license_age = license_age.days
        #    if license_age <= 0:
        #        license_age = 0
        #else:
        #    license_age = float('nan')
            
        time_since_last = current_inspection.inspection_date - previous_inspections.ix[last_inspection_index].inspection_date
        time_since_last = time_since_last.days
        
        prev_violations = previous_inspections.violations_count[last_inspection_index]
        prev_citations = previous_inspections.citations_count[last_inspection_index]
        prev_critical = previous_inspections.critical_count[last_inspection_index]
        prev_serious = previous_inspections.serious_count[last_inspection_index]
        prev_minor = previous_inspections.minor_count[last_inspection_index]
        prev_corrected = previous_inspections.corrected_count[last_inspection_index]

    else:
        fraction = float('nan')
        count = 0
        last = float('nan')
        license_age = float('nan')
        time_since_last = float('nan')
        prev_violations = float('nan')
        prev_citations = float('nan')
        prev_critical = float('nan')
        prev_serious = float('nan')
        prev_minor = float('nan')
        prev_corrected = float('nan')

    return np.array([count,
                     fraction,
                     last,
                     #license_age,
                     time_since_last,
                     prev_violations,
                     prev_citations,
                     prev_critical,
                     prev_serious,
                     prev_minor,
                     prev_corrected])

In [ ]:
#previous_inspection_features = np.zeros((len(data.index),11))

previous_inspection_features = np.zeros((len(data.index),10))

LOG_EVERY_N = 10000
for i in range(len(data.index)):
    index = data.index[i]
    previous_inspection_features[i,:] = prev_inspection_features(data,data.ix[index].inspection_id,data.ix[index].inspection_date)
    if (index % LOG_EVERY_N) == 0:
        print index

In [ ]:
data['previous_count'] = previous_inspection_features[:,0]
data['previous_fraction'] = previous_inspection_features[:,1]
data['previous_result'] = previous_inspection_features[:,2]
#data['license_age'] = previous_inspection_features[:,3]
data['time_since_last_inspection'] = previous_inspection_features[:,3]
data['previous_violations'] = previous_inspection_features[:,4]
data['previous_citations'] = previous_inspection_features[:,5]
data['previous_critical'] = previous_inspection_features[:,6]
data['previous_serious'] = previous_inspection_features[:,7]
data['previous_minor'] = previous_inspection_features[:,8]
data['previous_corrected'] = previous_inspection_features[:,9]

In [ ]:
for col in data.columns:
    print col

In [ ]:
data.to_csv('./data/data_built_features.csv',index=False)